# Fairify Artifact
Artifact of the paper "Fairify: Fairness Verification of Neural Networks" from ICSE 2023.

To access ChameleonCloud resources, you may need the account to log in to ChameleonCloud. You also need to have a project to allocate resources (e.g., node).

In [2]:
from chi import context

context.version = "1.0"

context.choose_site(default="CHI@TACC")
context.choose_project()

### Check available hardware

In [3]:
from chi import hardware

node_type = "compute_cascadelake_r"
available_nodes = hardware.get_nodes(node_type=node_type, filter_reserved=True)
if available_nodes:
    print(f"There currently are {len(available_nodes)} {node_type} nodes ready to use")
else:
    print(f"All {node_type} nodes are in use! You could use next_free_timeslot to see how long you need to wait, or use the calendar.")

There currently are 40 compute_cascadelake_r nodes ready to use


### Reserve node

In [4]:
from chi import lease
from datetime import timedelta
import os

my_lease = lease.Lease(f"{os.getenv('USER')}-power-management", duration=timedelta(hours=3))
my_lease.add_node_reservation(nodes=[available_nodes[0]]) # or you could use node_type=node_type
my_lease.add_fip_reservation(1) # include a floating ip
my_lease.submit(idempotent=True)

Waiting for lease to start...


Lease radhofanazizi_gmail_com-power-management has reached status active


### Create a server on the node

In [5]:
from chi import server

my_server = server.Server(
    f"{os.getenv('USER')}-power-management",
    reservation_id=my_lease.node_reservations[0]["id"],
    image_name="CC-Ubuntu22.04", # or use image_name
)
my_server.submit(idempotent=True)

Waiting for server radhofanazizi_gmail_com-power-management's status to become ACTIVE. This typically takes 10 minutes, but can take up to 20 minutes.


Server has moved to status ACTIVE


Attribute,radhofanazizi_gmail_com-power-management
Id,08d442b5-c62f-4976-9952-57d8c82989cc
Status,ACTIVE
Image Name,CC-Ubuntu22.04
Flavor Name,baremetal
Addresses,sharednet1: IP: 10.52.3.212 (v4) Type: fixed MAC: bc:97:e1:78:f1:a0
Network Name,sharednet1
Created At,2025-05-23T08:36:19Z
Keypair,trovi-28d289c
Reservation Id,29448623-5503-4c53-98af-6cd70a37a290
Host Id,b281b13a05d4a4d342f673906de4005142c2819a049809e34ac97306


### Configure networking on the node

In [6]:
fip = my_lease.get_reserved_floating_ips()[0]
my_server.associate_floating_ip(fip)
my_server.check_connectivity(host=fip)

Checking connectivity to 129.114.109.237 port 22.


Connection successful


# Setup nvidia drivers and cuda

Download drivers

In [ ]:
my_server.execute("""
sudo apt-get --purge remove -y '*cublas*' 'cuda*' 'nvidia-*' && \
sudo apt-get autoremove -y && \
sudo apt-get autoclean && \
sudo rm -rf /usr/local/cuda* && \
sudo apt update && \
sudo apt install -y nvidia-driver-470 && \
wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-keyring_1.0-1_all.deb && \
sudo dpkg -i cuda-keyring_1.0-1_all.deb && \
sudo apt update && \
sudo apt install -y cuda-toolkit-11-2
""")

Restart nvidia services

In [ ]:
my_server.execute("sudo modprobe -r nouveau")
my_server.execute("sudo rmmod nouveau 2>/dev/null || true")
my_server.execute("sudo update-initramfs -u")
my_server.execute("sudo modprobe nvidia")

Verifiy services

In [ ]:
my_server.execute("nvidia-smi")
my_server.execute("""
if [ -L /usr/local/bin/nvcc ]; then sudo rm /usr/local/bin/nvcc; fi && \
sudo ln -s /usr/local/cuda-11.2/bin/nvcc /usr/local/bin/nvcc
""")
my_server.execute("nvcc --version")

## Run Fairify

Now, we can finally run Fairify. First we need to clone the github repo first and then run the reprduce.sh script which contain the instructions from README.md
packaged into a bash file

In [12]:
my_server.execute("rm -rf Fairify && git clone https://github.com/radhofan/Fairify.git")

Cloning into 'Fairify'...


<Result cmd='rm -rf Fairify && git clone https://github.com/radhofan/Fairify.git' exited=0>

### Run dependencies and first experiment

In [13]:
my_server.execute("chmod +x Fairify/reproduce-experiment.sh")
my_server.execute("bash Fairify/reproduce-experiment.sh")

ERROR: File or directory already exists: '/home/cc/miniconda'
If you want to update an existing installation, use the -u option.


Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: done

# All requested packages already installed.

Running `shell init`, which:
 - modifies RC file: "/home/cc/.bashrc"
 - generates config for root prefix: "/home/cc/miniconda"
 - sets mamba executable to: "/home/cc/miniconda/bin/mamba"
The following has been added in your "/home/cc/.bashrc" file

# >>> mamba initialize >>>
# !! Contents within this block are managed by 'mamba shell init' !!
export MAMBA_EXE='/home/cc/miniconda/bin/mamba';
export MAMBA_ROOT_PREFIX='/home/cc/miniconda';
__mamba_setup="$("$MAMBA_EXE" shell hook --shell bash --root-prefix "$MAMBA_ROOT_PREFIX" 2> /dev/null)"
if [ $? -eq 0 ]; then
    eval "$__mamba_setup"
else
    alias mamba="$MAMBA_EXE"  # Fallback on help from mamba activate
fi
unset __mamba_setup
# <<< mamba initialize <<<



warning  libmamba 'repo.anaconda.com', a commercial channel hosted by Anaconda.com, is used.
    
warning  libmamba Please make sure you understand Anaconda Terms of Services.
    
warning  libmamba See: https://legal.anaconda.com/policies/en/




Transaction

  Prefix: /home/cc/miniconda/envs/fairify

  Updating specs:

   - python=3.9


  Package               Version  Build           Channel         Size
───────────────────────────────────────────────────────────────────────
  Install:
───────────────────────────────────────────────────────────────────────

  + _libgcc_mutex           0.1  main            pkgs/main     Cached
  + _openmp_mutex           5.1  1_gnu           pkgs/main     Cached
  + ca-certificates   2025.2.25  h06a4308_0      pkgs/main     Cached
  + ld_impl_linux-64       2.40  h12ee557_0      pkgs/main     Cached
  + libffi                3.4.4  h6a678d5_1      pkgs/main     Cached
  + libgcc-ng            11.2.0  h1234567_1      pkgs/main     Cached
  + libgomp              11.2.0  h1234567_1      pkgs/main     Cached
  + libstdcxx-ng         11.2.0  h1234567_1      pkgs/main     Cached
  + ncurses                 6.4  h6a678d5_0      pkgs/main     Cached
  + openssl              3.0.16  h5eee18b_0      

Reading package lists...
Building dependency tree...
Reading state information...
csvtool is already the newest version (2.4-1build3).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
Started running verification for Fairify/src/BM/Verify-BM-experiment.py models.


2025-05-23 09:25:12.179577: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-05-23 09:25:12.179597: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'
pip install 'aif360[inFairness]'
pip install 'aif360[Reductions]'
2025-05-23 09:25:13.656261: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2025-05-23 09:25:13.656280: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2025-05-23 09:25:13.656294: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on t

Missing Data: 10700 rows removed.
Number of partitions:  510
==================  STARTING MODEL BM-13.h5
###################
INTERVAL BASED PRUNING
SINGULAR VERIFICATION
Pruning done!
Verifying ...
unsat
V time:  5.906
******************
14 column
0       0
1       0
2       0
3       1
4       0
       ..
4569    1
4570    0
4571    0
4572    1
4573    0
Name: 14, Length: 4574, dtype: int64
y_true
0       0
1       0
2       1
3       1
4       0
       ..
4569    0
4570    0
4571    1
4572    0
4573    0
Length: 4574, dtype: int64
True: 600 | False: 3974
y_pred
0       False
1       False
2       False
3        True
4       False
        ...  
4569    False
4570    False
4571    False
4572    False
4573    False
Length: 4574, dtype: bool
True: 175 | False: 4399
prot_attr
0       0
1       0
2       0
3       1
4       0
       ..
4569    1
4570    0
4571    0
4572    1
4573    0
Length: 4574, dtype: int64
INTERVAL BASED PRUNING
SINGULAR VERIFICATION
Pruning done!
Verifying ...
unsat


<Result cmd='bash Fairify/reproduce-experiment.sh' exited=0>

Results for first experiment

In [14]:
my_server.execute("csvtool readable Fairify/src/BM/age-BM-13.csv | less -S")

Partition_ID Verification SAT_count UNSAT_count UNK_count h_attempt h_success B_compression S_compression ST_compression H_compression T_compression SV-time S-time HV-Time H-Time Total-Time C-check V-accurate Original-acc Pruned-acc Acc-dec C1                                                                                                                                                                                  C2
1            unsat        0         1           0         0         0         0.7647        0.0           0.7647         0             0.7647        5.906   6.08   0       0.0    6.08       0       0          0.8826       1.0        -                                                                                                                                                                                           
2            unsat        0         2           0         0         0         0.7647        0.0           0.7647         0             0.7647        3.528

<Result cmd='csvtool readable Fairify/src/BM/age-BM-13.csv | less -S' exited=0>

In [15]:
my_server.execute("csvtool readable Fairify/src/BM/age-counterexample-bank-empty.csv | less -S")

job marital education default housing loan contact month day_of_week emp.var.rate duration campaign pdays previous poutcome age
1   8       0         1       1       0    0       0     0           4            1309     0        50    0        1        1   1
0   8       0         1       1       0    0       0     0           4            1309     0        50    0        1        1   0
1   0       0         0       1       1    0       1     10          0            2704     1        50    296      7        1   1
0   0       0         0       1       1    0       1     10          0            2704     1        50    296      7        1   0
0   0       0         6       1       0    0       1     0           5            3320     0        1     906      7        0   0
1   0       0         6       1       0    0       1     0           5            3320     0        1     906      7        0   1
1   2       0         0       0       0    1       0     1           0            1944     1

<Result cmd='csvtool readable Fairify/src/BM/age-counterexample-bank-empty.csv | less -S' exited=0>

In [16]:
my_server.execute("csvtool readable Fairify/src/BM/age-synthetic-bank-predicted-BM-13-metrics.csv | less -S")

Partition ID Original Accuracy  Original F1 Score   Pruned Accuracy Pruned F1 DI                  SPD                  EOD                  AOD                  ERD                  CNT          TI
1            0.8825972890249235 0.30709677419354836 1.0             1.0       0.05222601966304822 -0.14863614082775076 -0.42251815980629537 -0.24463300393464102 -0.06690281517609131 [0.99029296] 0.11623188566049097
2            0.8825972890249235 0.30709677419354836 1.0             1.0       0.05222601966304822 -0.14863614082775076 -0.42251815980629537 -0.24463300393464102 -0.06690281517609131 [0.99029296] 0.11623188566049097
3            0.8825972890249235 0.30709677419354836 1.0             1.0       0.05222601966304822 -0.14863614082775076 -0.42251815980629537 -0.24463300393464102 -0.06690281517609131 [0.99029296] 0.11623188566049097
4            0.8825972890249235 0.30709677419354836 1.0             1.0       0.05222601966304822 -0.14863614082775076 -0.42251815980629537 -0.24463300393464

<Result cmd='csvtool readable Fairify/src/BM/age-synthetic-bank-predicted-BM-13-metrics.csv | less -S' exited=0>